In [7]:
import tensorflow as tf 
import os
import numpy as np
from PIL import Image
from tensorflow import keras
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.layers import Conv2D, MaxPooling2D, Input, ZeroPadding2D, Dropout, Conv2DTranspose, Cropping2D, concatenate, Lambda

In [8]:
from skimage.io import imread, imshow
from skimage.transform import resize
from tqdm import tqdm 

## Keras Method for data

In [11]:
X_train_path="train_frame"  #Original
Y_train_path="train_mask"  #Labels

X_train=ImageDataGenerator().flow_from_directory(X_train_path, class_mode=None)
Y_train=ImageDataGenerator().flow_from_directory(Y_train_path, class_mode=None)

Found 512 images belonging to 1 classes.
Found 512 images belonging to 1 classes.


## Model

In [13]:
img_height=512
img_width=512
channel=1
input_shape=(img_height,img_width,channel)
input_=tf.keras.layers.Input(shape=input_shape)
s = tf.keras.layers.Lambda(lambda x: x / 255)(input_)

In [14]:
c1 = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2), strides=2)(c1)

c2 = tf.keras.layers.Conv2D(filters=32,kernel_size= (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2),strides=2)(c2)
 
c3 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2),strides=2)(c3)

c4 = tf.keras.layers.Conv2D(filters=128,  kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Conv2D(filters=128,  kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D((2, 2),strides=2)(c4)


c5 = tf.keras.layers.Conv2D(filters=256,  kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Conv2D(filters=256,  kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

In [20]:
u1 = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=(2, 2), strides=(2, 2), padding='same')(c5)
u1 = tf.keras.layers.concatenate([u1, c4])
c6 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u1)
c6 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u2 = tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=(2, 2), strides=(2, 2), padding='same')(c6)
u2 = tf.keras.layers.concatenate([u2, c3])
c7 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u2)
c7 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

u3 = tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=(2, 2), strides=(2, 2), padding='same')(c7)
u3 = tf.keras.layers.concatenate([u3, c2])
c8 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u3)
c8 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

u4 = tf.keras.layers.Conv2DTranspose(filters=16, kernel_size=(2, 2), strides=(2, 2), padding='same')(c8)
u4 = tf.keras.layers.concatenate([u4, c1])
c9 = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u4)
c9 = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

In [21]:
out = tf.keras.layers.Conv2D(3, (1, 1), activation='sigmoid')(c9)
opt=tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=.1)

In [22]:
model = tf.keras.Model(inputs=input_, outputs=out)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 512, 512, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 16) 160         lambda[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 16) 2320        conv2d[0][0]                     
______________________________________________________________________________________________

In [23]:
model.fit(X_train, Y_train, batch_size=32, epochs=5)

ValueError: `y` argument is not supported when using `keras.utils.Sequence` as input.